<a href="https://colab.research.google.com/github/johc0205/python_api_practice/blob/main/callBuildingPermitApi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xmltodict

xml 태그 한글로 변경



In [2]:
import requests
from urllib.parse import unquote
import xmltodict
import csv
from google.colab import drive

# 드라이브 마운트
drive.mount('/content/drive')

def getBubJungdong(api_key: str):
    url = 'http://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList'

    params = {
        'serviceKey': api_key,
        'pageNo': '1',
        'numOfRows': '40',
        'locatadd_nm': '서울특별시 영등포구'
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()

        if response.text:
            xml_dict = xmltodict.parse(response.text)

            result_list = []
            for row in xml_dict.get('StanReginCd', {}).get('row', []):
                data_dict = {
                    'locatadd_nm': row.get('locatadd_nm'),
                    'locatjumin_cd': row.get('locatjumin_cd')
                }
                result_list.append(data_dict)
        return result_list

    except requests.exceptions.HTTPError as errh:
        print("HTTP 에러:", errh)
    except requests.exceptions.RequestException as err:
        print("에러:", err)

def getBuildingPermit(sigunCd: str, bubjungdong: str, api_key: str):
    url = "http://apis.data.go.kr/1613000/ArchPmsService_v2/getApBasisOulnInfo"

    params = {
        'ServiceKey': api_key,
        'sigunguCd': sigunCd,
        'bjdongCd': bubjungdong
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.text

    except requests.exceptions.HTTPError as errh:
        print("HTTP 에러:", errh)
    except requests.exceptions.RequestException as err:
        print("에러:", err)

# 번역할 영어-한글 매핑 딕셔너리
translation_mapping = {
    'guyukCd': '구역코드',
    'guyukCdNm': '구역코드명',
    'jimokCd': '지목코드',
    'jiyukCd': '지역코드',
    'jiguCd': '지구코드',
    'archGbCdNm': '건축구분코드',
    'archGbCd': '건축구분코드명',
    'platArea': '대지면적(㎡)',
    'archArea': '건축면적(㎡)',
    'bcRat': '건폐율(%)',
    'totArea': '연면적(㎡)',
    'vlRatEstmTotArea': '용적률산정연면적(㎡)',
    'vlRat': '용적률(%)',
    'mainBldCnt': '주건축물수',
    'atchBldDongCnt': '부속건축물동수',
    'mainPurpsCd': '주용도코드',
    'mainPurpsCdNm': '주용도코드명',
    'hhldCnt': '세대수(세대)',
    'hoCnt': '호수(호)',
    'fmlyCnt': '가구수(가구)',
    'totPkngCnt': '총주차수',
    'stcnsSchedDay': '착공예정일',
    'stcnsDelayDay': '착공연기일',
    'realStcnsDay': '실제착공일',
    'archPmsDay': '건축허가일',
    'useAprDay': '사용승인일',
    'crtnDay': '생성일자',
    'rnum': '순번',
    'platPlc': '대지위치',
    'sigunguCd': '시군구코드',
    'bjdongCd': '법정동코드',
    'platGbCd': '대지구분코드',
    'bun': '번',
    'ji': '지',
    'mgmPmsrgstPk': '관리허가대장PK',
    'bldNm': '건물명',
    'splotNm': '특수지명',
    'block': '블록',
    'lot': '로트',
    'jimokCdNm': '지목코드명',
    'jiyukCdNm': '지역코드명',
    'jiguCdNm': '지구코드명'
}

# API 키를 unquote 함수로 디코딩
api_key = unquote("INSERT YOUR KEY")
siGunGu = {}
bubJungDong = {}

result_list = []

# 함수 호출 및 결과 저장
seoulCodeList = getBubJungdong(api_key)
for sigungu in seoulCodeList:
    sigungu_impl = sigungu['locatadd_nm'].split()

    if len(sigungu_impl) == 2:
        siGunGu[sigungu_impl[1]] = sigungu['locatjumin_cd'][0:5]
    else:
        if siGunGu.get(sigungu_impl[1]) is None:
            siGunGu[sigungu_impl[1]] = sigungu['locatjumin_cd'][0:5]
        bubJungDong[sigungu_impl[2]] = sigungu['locatjumin_cd'][5:]

        # 건축인허가 정보 가져오기
        result = getBuildingPermit(siGunGu[sigungu_impl[1]], sigungu['locatjumin_cd'][5:], api_key)
        if result is not None:
            result_dict = xmltodict.parse(result)
            impl_list = []
            for k in (result_dict['response']['body']['items']['item']):
              translated_dict = {translation_mapping.get(key, key): value for key, value in k.items()}
              impl_list.append(translated_dict)
            result_dict['response']['body']['items']['item'] = impl_list
            result_list.extend(result_dict['response']['body']['items']['item'])
# CSV 파일에 쓰기
csv_file_path = "/content/drive/MyDrive/rpa_test/buildPermit_Yeongdeungpo.csv"

with open(csv_file_path, mode="w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)

    # 헤더 쓰기
    header_written = False

    # 결과값 쓰기
    for result_item in result_list:
        if not header_written:
            # 헤더 쓰기
            writer.writerow(result_item.keys())
            header_written = True

        # 결과값 쓰기
        writer.writerow(result_item.values())

print(f"CSV 파일이 {csv_file_path}에 생성되었습니다.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CSV 파일이 /content/drive/MyDrive/rpa_test/buildPermit_Yeongdeungpo.csv에 생성되었습니다.
